In [ ]:
import pandas as pd
df = pd.read_csv('/content/cleaned_responses(1).csv')

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=edf6f6fb41a2004ab17aa4f5ed9d752446984c4f9d19dffdddada42b2f834981
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load your CSV file
df = pd.read_csv('/content/cleaned_responses(1).csv')

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize BLEU smoothing function to avoid zero scores for n-gram overlap warnings
smoothing_function = SmoothingFunction().method1

# Prepare a function to calculate the scores
def calculate_scores(reference, hypothesis):
    # Check if hypothesis is valid (not NaN)
    if isinstance(hypothesis, float) and pd.isna(hypothesis):
        return {
            'BLEU': None,
            'METEOR': None,
            'ROUGE-L': None
        }

    # Tokenize reference and hypothesis for METEOR
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()

    # BLEU score with smoothing
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing_function)

    # METEOR expects tokenized text
    meteor = meteor_score([reference_tokens], hypothesis_tokens)

    # ROUGE score
    rouge = scorer.score(reference, hypothesis)

    # Extract ROUGE-L score
    rouge_l = rouge['rougeL'].fmeasure

    return {
        'BLEU': bleu,
        'METEOR': meteor,
        'ROUGE-L': rouge_l
    }

# Initialize variables to store the total scores
total_bleu = 0
total_meteor = 0
total_rouge_l = 0
valid_rows = 0

# Loop through the dataset and calculate scores
for idx, row in df.iterrows():
    reference = row['appropriate']
    llm_without_finetuning = row['Formatted_Response']

    scores = calculate_scores(reference, llm_without_finetuning)

    # Check if scores are valid before adding them to the totals
    if scores['BLEU'] is not None:
        total_bleu += scores['BLEU']
        total_meteor += scores['METEOR']
        total_rouge_l += scores['ROUGE-L']
        valid_rows += 1  # Increment the count of valid rows

# Calculate the average scores
average_bleu = total_bleu / valid_rows if valid_rows else 0
average_meteor = total_meteor / valid_rows if valid_rows else 0
average_rouge_l = total_rouge_l / valid_rows if valid_rows else 0

# Display the average scores
print("Average Scores:")
print("BLEU:", average_bleu)
print("METEOR:", average_meteor)
print("ROUGE-L:", average_rouge_l)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Average Scores:
BLEU: 0.22211917735729467
METEOR: 0.5255811784024979
ROUGE-L: 0.5970250113359516
